# 앞으로 객체 크롤링 진행햐아 하는 제품

In [2]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [8]:
need_to_crawl = []

In [ ]:
df_brd_w_prd_v3 = pd.read_csv("df_brd_w_prd_v3.csv")
df_brd_w_prd_v3 = df_brd_w_prd_v3.iloc[:, 1:]

In [3]:
v4_update = pd.read_csv("v4_update.csv")
v4_update = v4_update.iloc[:, 1:]

In [5]:
lst_v4_prd_link = list(v4_update['제품 link'])

In [ ]:
#현재까지 크롤링 완료되어 한꺼번에 붙인 df
df_glowpick_prd_info = pd.read_csv("df_glowpick_prd_info.csv")
df_glowpick_prd_info = df_glowpick_prd_info.iloc[: 1:]

In [7]:
lst_df_glowpick_prd_link = list(df_glowpick_prd_info['제품 link'])

NameError: name 'df_glowpick_prd_info' is not defined

In [ ]:
for i in range(len(df_brd_w_prd_v3)):
    for j in list(df_brd_w_prd_v3.loc[i, 'product_link']):
        if j not in lst_df_glowpick_prd_link and j not in lst_v4_prd_link:
            need_to_crawl.append(j)

---
# 이미지 src만 추가로 받아와야 하는 상품 img src만 받아오기

In [ ]:
#현재까지 크롤링 완료되어 한꺼번에 붙인 df
df_glowpick_prd_info = pd.read_csv("df_glowpick_prd_info.csv")
df_glowpick_prd_info = df_glowpick_prd_info.iloc[: 1:]

In [ ]:
def get_img_src(prd_link):
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    wd.get(prd_link)
    time.sleep(5)
    wd.maximize_window()
    
    socket.setdefaulttimeout(30)
        
    try:
        
        wd.find_element_by_xpath('//*[@id=\"default-layout\"]/div/div[1]/span/div/div[2]/div[2]/button[2]').click()
        time.sleep(5)

        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        
        time.sleep(3)
        img_src = wd.find_element_by_xpath("/html/body/div/div/div/div/main/div/section/div[1]/div/div/img").get_attribute('src')
        img_src = re.sub("w=[0-9]*", "w=500", img_src)
        wd.close()
        
        return img_src

    except Exception as e:
        exception.append(prd_link)
        print(e)

In [9]:
#df_glowpick_prd_info는 지금까지 크롤링 완료된 glowpick 상품 df

for i in list(df_glowpick_prd_info.index):
    prd_link = df_glowpick_prd_info[i, '제품 link']
    img_src = get_img_src(prd_link)
    df_glowpick_prd_info[i, 'img_src'] = ima_src

NameError: name 'df_glowpick_prd_info' is not defined